<a href="https://colab.research.google.com/github/JoaoMKlGui/ProjetoNeuralMind/blob/master/ChatBotUnicamp2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Baixando todas as dependências do projeto

In [ ]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab]
wget --no-check-certificate https://dl.xpdfreader.com/xpdf-tools-linux-4.04.tar.gz
tar -xvf xpdf-tools-linux-4.04.tar.gz && sudo cp xpdf-tools-linux-4.04/bin64/pdftotext /usr/local/bin

#Definindo as funções para a criação do retriever e a pipeline generativa

In [ ]:
from haystack.nodes import PromptNode, PromptTemplate, AnswerParser, BM25Retriever
from haystack.pipelines import Pipeline

def init_prompt():
    prompt_template = PromptTemplate(
        prompt="""Responda a questão de forma precisa baseado apenas no documento que lhe foi passado. Caso a informação não esteja no documento, diga que uma resposta não pode ser gerada apenas com as informações dadas.
        Documento:{join(documents)}
        Pergunta:{query}
        stop_words=["Observação:"],
        Resposta:
        """,
        output_parser=AnswerParser()
    )

    return prompt_template


def init_prompt_node(api_key_openai: str, model_name):
    prompt_node = PromptNode(
        model_name_or_path = model_name,
        api_key=api_key_openai,
        default_prompt_template=init_prompt()
    )

    return prompt_node

def init_retriever(document_store):
    retriever = BM25Retriever(
        document_store = document_store
    )

    return retriever



def init_generative_pipeline(document_store, open_ai_key):
    generative_pipeline = Pipeline()
    generative_pipeline.add_node(
        component = init_retriever(document_store),
        name = "retriever",
        inputs=["Query"]
    )
    generative_pipeline.add_node(
        component=init_prompt_node(open_ai_key, "text-davinci-003"),
        name = "prompt_node",
        inputs = ["retriever"]
    )

    return generative_pipeline

#Definindo a pipeline para transformação do pdf em um document do Haystack

In [ ]:
#pipeline para extrair infos das páginas web do regimento do vestibular unicamp 2024
'''
Installing external dependencies

Todas essas dependências estão presentes em "requirements.txt"

'''

from haystack.document_stores import InMemoryDocumentStore
from haystack.nodes import PDFToTextConverter

def init_document_store(document):
    doc_store = InMemoryDocumentStore(use_bm25 = True)
    doc_store.write_documents(document)

    return doc_store


def convert_pdf_to_text(path: str):
    """
    returns a document
    """
    conversor = PDFToTextConverter()

    documento_text = conversor.convert(file_path = path)

    return documento_text


#Agora, criando as funções para que o mecanismo de chat-bot funcione. O coração do código!

In [ ]:
from haystack.utils import print_answers
from haystack.agents import Tool
from haystack.agents import AgentStep, Agent
from haystack.agents.base import Agent, ToolsManager
from haystack.nodes import PromptNode
from haystack.agents.memory import ConversationSummaryMemory

prompt_agente = """
    Nesta conversa, o usuário humano interage com uma Inteligência Artificial, que será o agente. O usuário faz perguntas e o agente passa por diversos processamentos para retornar uma resposta bem informada ao usuário.
    O agente deve utilizar as ferramentas disponíveis para achar informações atualizadas e a resposta deve ser baseada somente na saída das ferramentas.
    O agente deve ignorar todo seu conhecimento prévio para responder a pergunta, utilizando somente as ferramentas passadas.
    O agente tem acesso a estas ferramentas:
    {tool_names_with_descriptions}

    A seguir, tem a conversa prévia entre humano e IA:
    {memory}

    A resposta do agente IA deve começar com algum destes:
    Pensamento: [O processo de raciocínio do agente]
    Ferramenta: [Nome das ferramentas] (em uma nova linha) Entrada da Ferramenta: [entrada como uma pergunta para a ferramenta selecionada SEM aspas e em uma nova linha] (Estes devem ser sempre fornecidos juntos e em linhas separadas.)
    Observação: [Resultado da ferramenta]
    Resposta Final: [Resposta final para a pergunta do humano]
    Ao selecionar uma ferramenta, o Agente deve fornecer o par "Ferramenta:" e "Entrada da ferramenta:" na mesma resposta, mas em linhas separadas.
    O Agente não deve solicitar informações, esclarecimentos ou contexto adicionais ao usuário humano.
    Se o Agente não conseguir encontrar uma resposta específica após esgotar as ferramentas e abordagens disponíveis, ele responderá com a Resposta Final: Inconclusivo, sinto muito.

    Pergunta: {query}
    Pensamento:
    {transcript}

"""

def init_search_tool(gen_pipeline):
    search_tool = Tool(
        name="busca_vest_unicamp_2024",
        pipeline_or_node=gen_pipeline,
        description="útil quando você precisa responder perguntas sobre o vestibular 2024 da unicamp",
        output_variable="respostas",
    )

    return search_tool

def resolucao(query, agent, agent_step):
    return {
        "query": query,
        "tool_names_with_descriptions": agent.tm.get_tool_names_with_descriptions(),
        "transcript": agent_step.transcript,
        "memory": agent.memory.load()
    }

def run_conversation(api_key, pdf): #essa função irá realizar a conversão e junção dos diferentes pedidos para gerar uma resposta do chat gpt
    document = convert_pdf_to_text(pdf)
    doc_store = init_document_store(document)
    gen_pipeline = init_generative_pipeline(doc_store, api_key)
    search_tool = init_search_tool(gen_pipeline)
    agent_prompt_node = PromptNode("gpt-3.5-turbo", api_key = api_key, max_length=128)
    memory_prompt_node = PromptNode(
    "philschmid/bart-large-cnn-samsum", max_length=128, model_kwargs={"task_name": "text2text-generation"}
    )
    memory = ConversationSummaryMemory(memory_prompt_node, prompt_template="{chat_transcript}")

    conversational_agent = Agent(
        agent_prompt_node,
        prompt_template=prompt_agente,
        prompt_parameters_resolver=resolucao,
        memory=memory,
        tools_manager=ToolsManager([init_search_tool(gen_pipeline)]),
    )

    while True:
        pergunta = input("Faça uma pergunta sobre o vestibular unicamp 2024!")
        response = conversational_agent.run(pergunta)
        print(response)


#Agora está tudo pronto!
Basta você fazer upload do pdf do vestibular 2024 no colab, colar o caminho dele em path e sua chave da Open AI

In [ ]:
path = r'coloque aqui o path'
open_ai_key = "colque aqui sua chave api da open ai"


run_conversation(open_ai_key, path)